# Mini Projeto 2 - Árvores de Decisão
Caio Moreira Gomes - cmg  
Izabela Melo - imcm  
Marco Aurelio - mafs3  
Michael Barney - mbgj  

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Iris Flower Dataset
![](http://)O conjunto de dados consiste em 50 amostras de cada uma das três espécies da Flor Iris (Iris Setosa, Iris virginica e Iris versicolor). Quatro características foram medidas em cada amostra: o comprimento e a largura das sépalas e pétalas, em centímetros.

Nosso objetivo será determinar a espécie da Iris a partir das 4 medidas apresentadas.

[Kaggle](https://www.kaggle.com/arshid/iris-flower-dataset)

## Análise
Primeiro precisamos entender melhor os dados apresentados e as suas relações.

In [ ]:
iris_df = pd.read_csv('/kaggle/input/iris-flower-dataset/IRIS.csv')
iris_df.head()

In [ ]:
iris_df['species'].value_counts()

In [ ]:
sns.pairplot(iris_df, hue = 'species')

## Árvore de Decisão
Iremos usar uma árvore de decisão para fazer a classificação.

In [ ]:
iris_y = iris_df['species']
iris_X = iris_df.drop(['species'], axis = 1)

Faremos a divisão de dados numa proporção 80/20, ou seja, 80% dos dados para treino e 20% para teste

In [ ]:
iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X,
                                                                        iris_y, 
                                                                        test_size = 0.2, 
                                                                        random_state = 42)

Inicialmente, usaremos os dados de treino para treinar nossa árvora de decisão:

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(iris_X_train, iris_y_train)

In [ ]:
iris_y_train_pred = tree.predict(iris_X_train)
iris_report_train = classification_report(iris_y_train, iris_y_train_pred)
print(iris_report_train)

Em seguida vamos usar os dados de teste:

In [ ]:
iris_y_pred = tree.predict(iris_X_test)
iris_report = classification_report(iris_y_test, iris_y_pred)
print(iris_report)

Vemos assim que estamos conseguindo prever a espécie da Íris com uma precisão de 100% através do nosso classificador.

# Titanic
A tarefa é simples, usar aprendizagem de máquina para criar um modelo que prevê quais passageiros sobreviveram ao naufrágio do Titanic.

Embora houvesse sorte envolvida na sobrevivência, alguns grupos de pessoas tinham mais probabilidade de sobreviver do que outros.

Neste desafio, construimos um modelo preditivo que responda à pergunta: "que tipo de pessoa tem mais probabilidade de sobreviver?" usando dados de passageiros (nome, idade, sexo, classe socioeconômica...)

[Kaggle](https://www.kaggle.com/c/titanic)

## Análise
Primeiro precisamos entender melhor os dados apresentados e as suas relações.

In [ ]:
titanic_train = pd.read_csv('/kaggle/input/titanic/train.csv')
titanic_train.head()

Vemos que a maioria das pessoas não sobreviveram (Survived = 0):

In [ ]:
titanic_train['Survived'].value_counts()

O campo **Embarked** representa o porto na qual a pessoa embarcou no Titanic (C = Cherbourg, Q = Queenstown, S = Southampton). Vemos que a maioria embarcou em Southampton:

In [ ]:
titanic_train['Embarked'].value_counts()

Para melhor analisar os dados, seria ideal utilizar valores numéricos. Assim, vamos alterar a coluna `Sex`, `Embarked` e `Cabin` para valores inteiros de forma que:   

**Sex**    
0 - homem   
1 - mulher 

**Embarked**    
0 - S   
1 - C    
2 - Q   

**Cabin**    
0 - Empty    
1 - Filled

In [ ]:
titanic_train['Sex'] = titanic_train['Sex'].map({'male': 0, 'female': 1})
titanic_train['Embarked'] = titanic_train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
titanic_train['Cabin'] = titanic_train["Cabin"].apply(lambda x: 0 if pd.isna(x) else 1)

titanic_train.head()

A coluna **SibSp** representa o número de irmãos e companheiros de cada passageiro, enquanto o **Parch** mostra o número de pais e filhos do passageiro. Podemos assim unificar estas duas colunas para que juntos representem o número de familiares:

In [ ]:
titanic_train['NumRelatives'] = titanic_train['SibSp'] + titanic_train['Parch']
titanic_train = titanic_train.drop(['SibSp', 'Parch'], axis = 1)
titanic_train.head()


Podemos também tirar algo útil a partir do nome do passageiro. Neste caso vamos apenas deixar o seu título (Mr, Miss, Mrs), sendo:

1 - Mr   
2 - Master  
3 - Mrs  
4 - Miss
5 - Other

In [ ]:
import re

def name_to_title(name):
    regex = re.search(' ([A-Za-z]+)\.', name)
    if regex:
        title = regex.group(1)
        title = title.replace('Mlle', 'Miss')
        title = title.replace('Ms', 'Miss')
        title = title.replace('Mme', 'Mrs')  
        return title
    return ""

titanic_train['Name'] = titanic_train["Name"].apply(name_to_title)
titanic_train['Name'] = titanic_train['Name'].map({"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4})
titanic_train['Name'] = titanic_train['Name'].fillna(0)

titanic_train.head()

Agora podemos tratar os valores que são nulos:

In [ ]:
titanic_train.isna().sum()

In [ ]:
titanic_train['Embarked'] = titanic_train['Embarked'].fillna(3) # outra opcao de embarked
titanic_train['Age'] = titanic_train['Age'].fillna(titanic_train['Age'].mean()) # outra opcao de embarked
titanic_train.head()


Vamos remover as colunas de identificação `PassengerId` e `Ticket`, pois não fornecem informações estatísticas.

In [ ]:
titanic_train = titanic_train.drop(['PassengerId', 'Ticket'], axis = 1)

Então nosso dataset final fica com essa cara:

In [ ]:
titanic_train.head()

In [ ]:
sns.pairplot(titanic_train, hue = 'Survived')

Com base nos gráficos que plotamos acima podemos tirar algumas conclusões iniciais:
* Passageiros da terceira classe morreram mais que todos os outros. A primeira classe foi a única que teve mais passageiros vivos do que mortos
* Passageiros do sexo feminino sobreviveram mais
* Passageiros mais novos tem uma leve vantagem 
* Quanto maior o valor da tarifa, maior a chance de viver (e isso tem ligação direta com o primeiro ponto)

## Árvore de Decisão
Iremos usar uma árvore de decisão para fazer a classificação.

In [ ]:
titanic_y_train = titanic_train['Survived']
titanic_X_train = titanic_train.drop(['Survived'], axis = 1)

In [ ]:
titanic_X_train, titanic_X_val, titanic_y_train, titanic_y_val = train_test_split(titanic_X_train,
                                                                                  titanic_y_train, 
                                                                                  test_size = 0.2, 
                                                                                  random_state = 42)

In [ ]:
titanic_tree = DecisionTreeClassifier(max_depth = 3)
titanic_tree.fit(titanic_X_train, titanic_y_train)

In [ ]:
titanic_y_pred = titanic_tree.predict(titanic_X_val)
print(classification_report(titanic_y_val, titanic_y_pred))

Vemos que conseguimos uma precisão de 82% a partir do nosso treinamento. Podemos até enxergar como ficou nossa arvore:

In [ ]:
from IPython.display import Image
from subprocess import check_call

# Exportar o modelo
with open("arvore.dot", 'w') as f:
     f = export_graphviz(titanic_tree,
                          out_file=f,
                          max_depth = 3,
                          impurity = True,
                          feature_names = list(titanic_train.drop(['Survived'], axis=1)),
                          class_names = ['Died', 'Survived'],
                          rounded = True,
                          filled= True )
        
# Converter de .dot para .png
check_call(['dot','-Tpng','arvore.dot','-o','arvore.png'])

Image("arvore.png")


Agora, podemos rodar nosso classificador nos dados de teste:

In [ ]:
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

titanic_test['Sex'] = titanic_test['Sex'].map({'male': 0, 'female': 1})
titanic_test['Embarked'] = titanic_test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
titanic_test['Cabin'] = titanic_test["Cabin"].apply(lambda x: 0 if pd.isna(x) else 1)

titanic_test['NumRelatives'] = titanic_test['SibSp'] + titanic_test['Parch']

titanic_test['Name'] = titanic_test["Name"].apply(name_to_title)
titanic_test['Name'] = titanic_test['Name'].map({"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Other": 5})
titanic_test['Name'] = titanic_test['Name'].fillna(0)


titanic_test['Embarked'] = titanic_test['Embarked'].fillna(3) # outra opcao de embarked
titanic_test['Age'] = titanic_test['Age'].fillna(titanic_test['Age'].mean()) # outra opcao de embarked

titanic_test = titanic_test.drop(['SibSp', 'Parch', 'PassengerId', 'Ticket'], axis = 1)

titanic_test['Fare'] = titanic_test['Fare'].fillna(0)

titanic_test.head()


In [ ]:
titanic_y_pred_test = titanic_tree.predict(titanic_test)
print(classification_report(titanic_y_val, titanic_y_pred))

Vemos assim que nosso classificador continua com uma média de precisão de 82% na detecção. Como houve um enorme fator de sorte envolvido no acidente, torna-se incrível que mesmo assim conseguimos definir se alguém sobreviveu a partir somente de dados básicos do passageiro com uma alta taxa de probabilidade.